In [ ]:
# Redisual attention model

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, GlobalAveragePooling2D, Activation, Conv2D
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
from glob import glob

In [ ]:
# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Define the image size
IMAGE_SIZE = (224, 224)

In [ ]:
# Set the paths to the training and validation data
train_path = 'chest_xray/train'
valid_path = 'chest_xray/test'

In [ ]:
def residual_attention_block(input_layer, filters):
  # First Convolution
  conv1 = Conv2D(filters, (3, 3), padding='same')(input_layer)
  conv1 = BatchNormalization()(conv1)
  conv1 = Activation('relu')(conv1)
   
  # Second Convolution
  conv2 = Conv2D(filters, (3, 3), padding='same')(conv1)
  conv2 = BatchNormalization()(conv2)
  conv2 = Activation('relu')(conv2)
   
  # Attention Mechanism
  attention = Conv2D(filters, (1, 1), activation='sigmoid', padding='same')(conv2)
  scaled = Multiply()([conv2, attention])
   
  # Residual Connection
  residual = Add()([input_layer, scaled])
   
  return residual

In [ ]:
def build_resatt_model(input_shape, num_classes, learning_rate=0.001):
  input_layer = Input(shape=input_shape)

  # Initial Convolution
  x = Conv2D(64, (7, 7), padding='same', strides=(2, 2))(input_layer)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
   
  # Max Pooling
  x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

  # Residual Attention Blocks
  for _ in range(3): # You can adjust the number of blocks
    x = residual_attention_block(x, 64)
   
  # Global Average Pooling Layer
  x = GlobalAveragePooling2D()(x)

  # Output layer
  output_layer = Dense(num_classes, activation='softmax')(x)

  model = Model(inputs=input_layer, outputs=output_layer)
  model.compile(
    loss='categorical_crossentropy', # Adjust for your problem
    optimizer=Adam(learning_rate),
    metrics=['accuracy']
  )
  return model


In [ ]:
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Multiply, Add




In [ ]:
# Create the ResAtt model
model = build_resatt_model(IMAGE_SIZE + (3,), num_classes=len(glob('chest_xray/train/*')))


In [ ]:
# View the model summary
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_10 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_27 (Conv2D)          (None, 112, 112, 64)         9472      ['input_10[0][0]']            
                                                                                                  
 batch_normalization_18 (Ba  (None, 112, 112, 64)         256       ['conv2d_27[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_19 (Activation)  (None, 112, 112, 64)         0         ['batch_normalization_18

In [ ]:
# Create image data generators for the training and validation data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
# Generate training and validation data
training_set = train_datagen.flow_from_directory('chest_xray/train',
                                                 target_size=IMAGE_SIZE,
                                                 batch_size=10,
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory('chest_xray/test',
                                            target_size=IMAGE_SIZE,
                                            batch_size=10,
                                            class_mode='categorical')


Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
# Train the model
model.fit(
    training_set,
    validation_data=test_set,
    epochs=4,  # You can adjust the number of epochs
)

Epoch 1/4
522/522 [==============================] - 183s 347ms/step - loss: 0.3233 - accuracy: 0.8545 - val_loss: 2.6269 - val_accuracy: 0.6250
Epoch 2/4
522/522 [==============================] - 258s 495ms/step - loss: 0.2596 - accuracy: 0.8909 - val_loss: 2.3597 - val_accuracy: 0.6250
Epoch 3/4
522/522 [==============================] - 8362s 16s/step - loss: 0.2289 - accuracy: 0.9051 - val_loss: 6.8861 - val_accuracy: 0.3878
Epoch 4/4
522/522 [==============================] - 188s 360ms/step - loss: 0.2082 - accuracy: 0.9197 - val_loss: 0.6236 - val_accuracy: 0.6987


Squeezenet Model

In [14]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Conv2D, MaxPooling2D, Activation, GlobalAveragePooling2D, Flatten, Dense
from keras.models import Model
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns
from sklearn.metrics import confusion_matrix
np.random.seed(42)
tf.random.set_seed(42)


In [15]:
# Define the image size
IMAGE_SIZE = (224, 224)

In [16]:
# Set the paths to the training and validation data
train_path = 'chest_xray/train'
valid_path = 'chest_xray/test'

In [19]:
# Define the number of classes
num_classes = 2  # Replace with the actual number of classes in your dataset


In [20]:
# Create a custom SqueezeNet model
def custom_squeezenet(input_shape, num_classes):
    input_tensor = Input(shape=input_shape)
    
    # Define SqueezeNet architecture here
    x = Conv2D(64, (3, 3), strides=(2, 2), activation='relu')(input_tensor)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    # Apply Global Average Pooling directly
    x = GlobalAveragePooling2D()(x)
    
    # Fully connected layers
    x = Dense(1024, activation='relu')(x)
    
    # Output layer
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=input_tensor, outputs=predictions)
    
    return model

In [21]:
# Create the custom SqueezeNet model
model = custom_squeezenet(input_shape=(224, 224, 3), num_classes=num_classes)


In [22]:
# Load the pre-trained weights (you need to specify the correct weights file)
model.load_weights('path_to_squeezenet_weights.h5')


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'path_to_squeezenet_weights.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [24]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from keras.layers import Input, Dense, Flatten
from keras.models import Model
from keras.applications.nasnet import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns
from sklearn.metrics import confusion_matrix


In [25]:
np.random.seed(42)
tf.random.set_seed(42)

In [26]:
# Define the image size
IMAGE_SIZE = [331, 331]  # PNASNetLarge input size


In [27]:
# Set the paths to the training and validation data
train_path = 'chest_xray/train'
valid_path = 'chest_xray/test'

In [33]:
# Load the PNASNetLarge model from TensorFlow Hub
pnasnet_url = "https://tfhub.dev/google/imagenet/pnasnet_large/classification/5"
pnasnet = hub.KerasLayer(pnasnet_url, trainable=False)


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)>

In [29]:
pip install --upgrade certifi



[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [32]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


In [35]:
# Import necessary libraries
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Flatten, Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [36]:
# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [37]:
# Define the image size
IMAGE_SIZE = [224, 224]

In [38]:
# Set the paths to the training and validation data
train_path = 'chest_xray/train'
valid_path = 'chest_xray/test'